In [ ]:
# import pandas as pd
# tin = pd.read_excel('./mineral_data.xlsx', sheet_name='Tin')
# tin.to_csv('./tin_data.csv', index=False)
# copper = pd.read_excel('./mineral_data.xlsx', sheet_name='Copper')
# copper.to_csv('./copper_data.csv', index=False)
# phosphorus = pd.read_excel('./mineral_data.xlsx', sheet_name='Phosphorus')
# phosphorus.to_csv('./phosphorus_data.csv', index=False)
# grappite = pd.read_excel('./mineral_data.xlsx', sheet_name='Graphite')
# grappite.to_csv('./graphite_data.csv', index=False)



### Graphite Visualization

In [7]:
import pandas as pd

df = pd.read_csv("./graphite_data.csv")
df['Year'] = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
df


,Monthly_Production,State of Production,Yearly_Export_Import(dollar),Year
0,[6739000 8264000 5168000 2043000 1942000 20120...,"{'Chhattisgarh': array([0, 0, 0, 0, 0, 0, 0, 0...",[40.01 120.03999999999999],2017
1,[2514000 2650000 2540000 2699000 3603000 27040...,"{'Chhattisgarh': array([0, 0, 0, 0, 0, 0, 0, 0...",[54.41 129.15],2018
2,[2077000 3328000 4838000 7217000 6917000 57740...,"{'Chhattisgarh': array([0, 0, 0, 0, 0, 0, 0, 0...",[50.6 95.28999999999999],2019
3,[7053000 6963000 3255000 214000 2334000 645400...,"{'Chhattisgarh': array([0, 0, 0, 0, 0, 0, 0, 0...",[38.62 97.64999999999999],2020
4,[6715000 7503000 8355000 8080000 4742000 65700...,"{'Chhattisgarh': array([490000, 545000, 730000...",[55.470000000000006 144.64],2021
5,[7304000 5057000 21980000 13601000 12127000 12...,"{'Chhattisgarh': array([0, 0, 0, 0, 0, 0, 0, 0...",[53.04 185.6],2022
6,[12840000 23068000 21283000 12859000 17044000 ...,"{'Chhattisgarh': array([0, 0, 0, 0, 0, 612000,...",[59.96 165.53],2023
7,[24142000 29441000 44336000 6282000 8863000 70...,"{'Chhattisgarh': array([0, 627000, 0, 0, 0, 0,...",[62.33 146.83],2024
8,[13882000 13256000 15778000 11424000 13882000 ...,"{'Chhattisgarh': array([0, 0, 0, 0, 0, 0, 0, 0...",[],2025


In [8]:
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from plotly.subplots import make_subplots

years = sorted(df["Year"].unique())

year_dropdown = widgets.Dropdown(
    options=years,
    value=years[0],
    description="Year:",
)

out1 = widgets.Output()

def update_year_plots(year):
    def state_change(change):
        if change["type"] == "change" and change["name"] == "value":
            with out1:
                out1.clear_output()
                fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Import, Export {year}", f"Monthly Production {year}"))

                fig.add_trace(
                    go.Bar(x=["Import", "Export"], y=[total_import, total_export], name="Trade"),
                    row=1, col=1
                )

                fig.add_trace(
                    go.Scatter(
                        x=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],
                        y=total_production,
                        mode="markers+text",
                        name="Production",
                        fill='tozeroy',
                        fillcolor="rgba(100, 200, 100, 0.3)"
                    ),
                    row=1, col=2
                )

                fig.update_yaxes(title_text="Quantity", row=1, col=1)
                fig.update_yaxes(title_text="Quantity", row=1, col=2)
                fig.update_layout(height=450, showlegend=False)

                fig.show()
                display(state_dropdown)
                fig3 = go.Figure()
                state = change["new"]
                monthly_state_production = result[state]
                # print(monthly_state_production)
                fig3.add_scatter(
                    x=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
                    y=monthly_state_production,
                    mode="markers+text",
                    name="State Production",
                    fill='tozeroy',
                )
                fig3.update_layout(
                    title=f"Monthly Production in {state} for {year}",
                    yaxis_title="Quantity",
                    height=450,
                )
                fig3.show()
    


    out1.clear_output()
    data_year = df[df["Year"] == year]
    # Handle empty or missing export/import data
    export_import_str = data_year['Yearly_Export_Import(dollar)'].values[0]
    export_import = np.fromstring(export_import_str.strip('[]'), sep=' ')
    result = eval(
    data_year['State of Production'].values[0],
    {"array": np.array, "object": object, "__builtins__": {}}
    )
    state_dropdown = widgets.Dropdown(
    options=list(result.keys()),
    value=list(result.keys())[1],
    description="State:",
    )
    state_dropdown.observe(state_change, names="value")
    if export_import.size == 2:
        total_export, total_import = export_import[0], export_import[1]
    else:
        total_export, total_import = 0, 0

    # Handle monthly production
    prod_str = data_year["Monthly_Production"].values[0]
    total_production = np.fromstring(prod_str.strip('[]'), sep=' ')

    with out1:
        fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Import, Export {year}", f"Monthly Production {year}"))

        fig.add_trace(
            go.Bar(x=["Import", "Export"], y=[total_import, total_export], name="Trade"),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],
                y=total_production,
                mode="markers+text",
                name="Production",
                fill='tozeroy',
                fillcolor="rgba(100, 200, 100, 0.3)"
            ),
            row=1, col=2
        )

        fig.update_yaxes(title_text="Quantity", row=1, col=1)
        fig.update_yaxes(title_text="Quantity", row=1, col=2)
        fig.update_layout(height=450, showlegend=False)

        fig.show()
        display(state_dropdown)
        fig3 = go.Figure()
        state = state_dropdown.value
        monthly_state_production = result[state]
        
        fig3.add_scatter(
            x=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
            y=monthly_state_production,
            mode="markers+text",
            name="State Production",
            fill='tozeroy',
            # fillcolor="rgba(100, 200, 100, 0.3)"
        )
        fig3.update_layout(
            title=f"Monthly Production in {state} for {year}",
            yaxis_title="Quantity",
            height=450,
        )
        fig3.show()


def on_year_change(change):
    update_year_plots(change["new"])

year_dropdown.observe(on_year_change, names="value")
display(widgets.HTML("<h2>Graphite Data Visualization Dashboard</h2>"))
display(year_dropdown, out1)
update_year_plots(years[0])


HTML(value='<h2>Graphite Data Visualization Dashboard</h2>')

Dropdown(description='Year:', options=(np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int6…

Output()

### Phoshphorus visualization

In [90]:
import pandas as pd

df = pd.read_csv("./phosphorus_data.csv")
df['Year'] = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
df

,Monthly_Production,State of Production,Yearly_Export_Import(dollar),Year
0,[200572000 236919000 1028360000 76298000 13906...,"{'Madhya Pradesh': array([3014000, 6710000, 84...",[2.89 0.8],2017
1,[489261000 269747000 407078000 76545000 309726...,"{'Madhya Pradesh': array([15461000, 12325000, ...",[3.3 0.96],2018
2,[316704000 432048000 473384000 224312000 34307...,"{'Madhya Pradesh': array([7063000, 8487000, 0,...",[3.7199999999999998 0.3],2019
3,[449227000 386387000 331190000 6053000 3105720...,"{'Madhya Pradesh': array([12596000, 3769000, 6...",[4.5 0.66],2020
4,[557146000 381585000 604438000 540430000 27879...,"{'Madhya Pradesh': array([2228000, 2134000, 22...",[11.02 0.47],2021
5,[736246000 669535000 1021331000 742125000 1015...,"{'Madhya Pradesh': array([9696000, 15397000, 2...",[12.37 0.09],2022
6,[1301830000 939869000 1226484000 968656000 117...,"{'Madhya Pradesh': array([48500000, 53618000, ...",[8.88 0.16],2023
7,[919077000 1592653000 1577942000 360802000 913...,"{'Madhya Pradesh': array([46825000, 39667000, ...",[8.22 0.49],2024
8,[1301709000 1120221000 788079000 811508000 130...,"{'Madhya Pradesh': array([82600000, 82878000, ...",[],2025


In [91]:
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from plotly.subplots import make_subplots

years = sorted(df["Year"].unique())

year_dropdown = widgets.Dropdown(
    options=years,
    value=years[0],
    description="Year:",
)

out1 = widgets.Output()

def update_year_plots(year):
    def state_change(change):
        if change["type"] == "change" and change["name"] == "value":
            with out1:
                out1.clear_output()
                fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Import, Export {year}", f"Monthly Production {year}"))

                fig.add_trace(
                    go.Bar(x=["Import", "Export"], y=[total_import, total_export], name="Trade"),
                    row=1, col=1
                )

                fig.add_trace(
                    go.Scatter(
                        x=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],
                        y=total_production,
                        mode="markers+text",
                        name="Production",
                        fill='tozeroy',
                        fillcolor="rgba(100, 200, 100, 0.3)"
                    ),
                    row=1, col=2
                )

                fig.update_yaxes(title_text="Quantity", row=1, col=1)
                fig.update_yaxes(title_text="Quantity", row=1, col=2)
                fig.update_layout(height=450, showlegend=False)

                fig.show()
                display(state_dropdown)
                fig3 = go.Figure()
                state = change["new"]
                monthly_state_production = result[state]
                # print(monthly_state_production)
                fig3.add_scatter(
                    x=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
                    y=monthly_state_production,
                    mode="markers+text",
                    name="State Production",
                    fill='tozeroy',
                )
                fig3.update_layout(
                    title=f"Monthly Production in {state} for {year}",
                    yaxis_title="Quantity",
                    height=450,
                )
                fig3.show()
    


    out1.clear_output()
    data_year = df[df["Year"] == year]
    # Handle empty or missing export/import data
    export_import_str = data_year['Yearly_Export_Import(dollar)'].values[0]
    export_import = np.fromstring(export_import_str.strip('[]'), sep=' ')
    result = eval(
    data_year['State of Production'].values[0],
    {"array": np.array, "object": object, "__builtins__": {}}
    )
    state_dropdown = widgets.Dropdown(
    options=list(result.keys()),
    value=list(result.keys())[1],
    description="State:",
    )
    state_dropdown.observe(state_change, names="value")
    if export_import.size == 2:
        total_export, total_import = export_import[0], export_import[1]
    else:
        total_export, total_import = 0, 0

    # Handle monthly production
    prod_str = data_year["Monthly_Production"].values[0]
    total_production = np.fromstring(prod_str.strip('[]'), sep=' ')

    with out1:
        fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Import, Export {year}", f"Monthly Production {year}"))

        fig.add_trace(
            go.Bar(x=["Import", "Export"], y=[total_import, total_export], name="Trade"),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],
                y=total_production,
                mode="markers+text",
                name="Production",
                fill='tozeroy',
                fillcolor="rgba(100, 200, 100, 0.3)"
            ),
            row=1, col=2
        )

        fig.update_yaxes(title_text="Quantity", row=1, col=1)
        fig.update_yaxes(title_text="Quantity", row=1, col=2)
        fig.update_layout(height=450, showlegend=False)

        fig.show()
        display(state_dropdown)
        fig3 = go.Figure()
        state = state_dropdown.value
        monthly_state_production = result[state]
        
        fig3.add_scatter(
            x=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
            y=monthly_state_production,
            mode="markers+text",
            name="State Production",
            fill='tozeroy',
            # fillcolor="rgba(100, 200, 100, 0.3)"
        )
        fig3.update_layout(
            title=f"Monthly Production in {state} for {year}",
            yaxis_title="Quantity",
            height=450,
        )
        fig3.show()


def on_year_change(change):
    update_year_plots(change["new"])

year_dropdown.observe(on_year_change, names="value")
display(widgets.HTML("<h2>Phosphorus Data Visualization Dashboard</h2>"))
display(year_dropdown, out1)
update_year_plots(years[0])


HTML(value='<h2>Phosphorus Data Visualization Dashboard</h2>')

Dropdown(description='Year:', options=(np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int6…

Output()

### TIN

In [ ]:
import pandas as pd

df = pd.read_csv("./tin_data.csv")
df['Year'] = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
df

,Monthly_Production,State of Production,Yearly_Export_Import(dollar),Year
0,[702000 449000 408000 726000 616000 303000 457...,"{'Chhattisgarh': array([702000, 449000, 408000...",[11.079999999999998 243.86],2017
1,[1059000 993000 584000 567000 466000 351000 52...,"{'Chhattisgarh': array([1059000, 993000, 58400...",[9.82 223.70000000000002],2018
2,[1771000 1929000 1159000 633000 487000 1115000...,"{'Chhattisgarh': array([1771000, 1929000, 1159...",[9.809999999999999 212.08999999999997],2019
3,[900000 836000 359000 288000 283000 278000 246...,"{'Chhattisgarh': array([836000, 900000, 359000...",[9.93 197.92],2020
4,[1267000 1015000 1860000 480000 355000 319000 ...,"{'Chhattisgarh': array([1267000, 1015000, 1860...",[19.32 379.86],2021
5,[6280000 8094000 5201000 3174000 3061000 35640...,"{'Chhattisgarh': array([6280000, 8094000, 5201...",[11.41 400.4],2022
6,[3423000 2628000 3318000 1462000 1591000 17030...,"{'Chhattisgarh': array([3423000, 2628000, 3318...",[17.29 339.12999999999994],2023
7,[1229000 126000 250000 184000 280000 296000 35...,"{'Chhattisgarh': array([1229000, 126000, 25000...",[22.86 494.33],2024
8,[3559000 2972000 1038000 468000 3559000 297200...,"{'Chhattisgarh': array([3559000, 2972000, 1362...",[],2025


In [ ]:

import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from plotly.subplots import make_subplots

years = sorted(df["Year"].unique())

year_dropdown = widgets.Dropdown(
    options=years,
    value=years[0],
    description="Year:",
)

out1 = widgets.Output()

def update_year_plots(year):
    def state_change(change):
        if change["type"] == "change" and change["name"] == "value":
            with out1:
                out1.clear_output()
                fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Import, Export {year}", f"Monthly Production {year}"))

                fig.add_trace(
                    go.Bar(x=["Import", "Export"], y=[total_import, total_export], name="Trade"),
                    row=1, col=1
                )

                fig.add_trace(
                    go.Scatter(
                        x=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],
                        y=total_production,
                        mode="markers+text",
                        name="Production",
                        fill='tozeroy',
                        fillcolor="rgba(100, 200, 100, 0.3)"
                    ),
                    row=1, col=2
                )

                fig.update_yaxes(title_text="Quantity", row=1, col=1)
                fig.update_yaxes(title_text="Quantity", row=1, col=2)
                fig.update_layout(height=450, showlegend=False)

                fig.show()
                display(state_dropdown)
                fig3 = go.Figure()
                state = change["new"]
                monthly_state_production = result[state]
                # print(monthly_state_production)
                fig3.add_scatter(
                    x=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
                    y=monthly_state_production,
                    mode="markers+text",
                    name="State Production",
                    fill='tozeroy',
                )
                fig3.update_layout(
                    title=f"Monthly Production in {state} for {year}",
                    yaxis_title="Quantity",
                    height=450,
                )
                fig3.show()
    


    out1.clear_output()
    data_year = df[df["Year"] == year]
    # Handle empty or missing export/import data
    export_import_str = data_year['Yearly_Export_Import(dollar)'].values[0]
    export_import = np.fromstring(export_import_str.strip('[]'), sep=' ')
    result = eval(
    data_year['State of Production'].values[0],
    {"array": np.array, "object": object, "__builtins__": {}}
    )
    state_dropdown = widgets.Dropdown(
    options=list(result.keys()),
    value=list(result.keys())[0],
    description="State:",
    )
    state_dropdown.observe(state_change, names="value")
    if export_import.size == 2:
        total_export, total_import = export_import[0], export_import[1]
    else:
        total_export, total_import = 0, 0

    # Handle monthly production
    prod_str = data_year["Monthly_Production"].values[0]
    total_production = np.fromstring(prod_str.strip('[]'), sep=' ')

    with out1:
        fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Import, Export {year}", f"Monthly Production {year}"))

        fig.add_trace(
            go.Bar(x=["Import", "Export"], y=[total_import, total_export], name="Trade"),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],
                y=total_production,
                mode="markers+text",
                name="Production",
                fill='tozeroy',
                fillcolor="rgba(100, 200, 100, 0.3)"
            ),
            row=1, col=2
        )

        fig.update_yaxes(title_text="Quantity", row=1, col=1)
        fig.update_yaxes(title_text="Quantity", row=1, col=2)
        fig.update_layout(height=450, showlegend=False)

        fig.show()
        display(state_dropdown)
        fig3 = go.Figure()
        state = state_dropdown.value
        monthly_state_production = result[state]
        
        fig3.add_scatter(
            x=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
            y=monthly_state_production,
            mode="markers+text",
            name="State Production",
            fill='tozeroy',
            # fillcolor="rgba(100, 200, 100, 0.3)"
        )
        fig3.update_layout(
            title=f"Monthly Production in {state} for {year}",
            yaxis_title="Quantity",
            height=450,
        )
        fig3.show()


def on_year_change(change):
    update_year_plots(change["new"])

year_dropdown.observe(on_year_change, names="value")
display(widgets.HTML("<h2>Phosphorus Data Visualization Dashboard</h2>"))
display(year_dropdown, out1)
update_year_plots(years[0])


HTML(value='<h2>Phosphorus Data Visualization Dashboard</h2>')

Dropdown(description='Year:', options=(np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int6…

Output()

### Copper

In [5]:
import pandas as pd

df = pd.read_csv("./copper_data.csv")
df['Year'] = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
df

,Monthly_Production,State of Production,Yearly_Export_Import(dollar),Year
0,[677973000 563372000 722886000 458581000 55843...,"{'Madhya Pradesh': array([303082000, 265812000...",[3481.36 4575.12],2017
1,[768973000 684118000 863190000 711094000 10365...,"{'Madhya Pradesh': array([313348000, 266510000...",[1067.35 5346.72],2018
2,[700603000 738627000 795379000 726188000 65035...,"{'Madhya Pradesh': array([331077000, 340419000...",[919.05 5147.09],2019
3,[779138000 676714000 605841000 196068000 58744...,"{'Madhya Pradesh': array([442230000, 396012000...",[1444.11 4665.36],2020
4,[922667000 954018000 971543000 809168000 97543...,"{'Madhya Pradesh': array([494901000, 522158000...",[2429.56 7114.07],2021
5,[1116283000 980117000 993978000 851084000 8896...,"{'Madhya Pradesh': array([570379000, 528841000...",[1848.54 7688.41],2022
6,[939495000 912198000 1330054000 912575000 9710...,"{'Madhya Pradesh': array([481123000, 505384000...",[2526.23 9294.75],2023
7,[1339091000 1166231000 1491796000 1204970000 1...,"{'Madhya Pradesh': array([724136000, 598834000...",[2320.58 10806.22],2024
8,[942199000 794873000 1105404000 763917000 9421...,"{'Madhya Pradesh': array([729924000, 611058000...",[],2025


In [6]:
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from plotly.subplots import make_subplots

years = sorted(df["Year"].unique())

year_dropdown = widgets.Dropdown(
    options=years,
    value=years[0],
    description="Year:",
)

out1 = widgets.Output()

def update_year_plots(year):
    def state_change(change):
        if change["type"] == "change" and change["name"] == "value":
            with out1:
                out1.clear_output()
                fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Import, Export {year}", f"Monthly Production {year}"))

                fig.add_trace(
                    go.Bar(x=["Import", "Export"], y=[total_import, total_export], name="Trade"),
                    row=1, col=1
                )

                fig.add_trace(
                    go.Scatter(
                        x=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],
                        y=total_production,
                        mode="markers+text",
                        name="Production",
                        fill='tozeroy',
                        fillcolor="rgba(100, 200, 100, 0.3)"
                    ),
                    row=1, col=2
                )

                fig.update_yaxes(title_text="Quantity", row=1, col=1)
                fig.update_yaxes(title_text="Quantity", row=1, col=2)
                fig.update_layout(height=450, showlegend=False)

                fig.show()
                display(state_dropdown)
                fig3 = go.Figure()
                state = change["new"]
                monthly_state_production = result[state]
                # print(monthly_state_production)
                fig3.add_scatter(
                    x=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
                    y=monthly_state_production,
                    mode="markers+text",
                    name="State Production",
                    fill='tozeroy',
                )
                fig3.update_layout(
                    title=f"Monthly Production in {state} for {year}",
                    yaxis_title="Quantity",
                    height=450,
                )
                fig3.show()
    


    out1.clear_output()
    data_year = df[df["Year"] == year]
    # Handle empty or missing export/import data
    export_import_str = data_year['Yearly_Export_Import(dollar)'].values[0]
    export_import = np.fromstring(export_import_str.strip('[]'), sep=' ')
    result = eval(
    data_year['State of Production'].values[0],
    {"array": np.array, "object": object, "__builtins__": {}}
    )
    state_dropdown = widgets.Dropdown(
    options=list(result.keys()),
    value=list(result.keys())[1],
    description="State:",
    )
    state_dropdown.observe(state_change, names="value")
    if export_import.size == 2:
        total_export, total_import = export_import[0], export_import[1]
    else:
        total_export, total_import = 0, 0

    # Handle monthly production
    prod_str = data_year["Monthly_Production"].values[0]
    total_production = np.fromstring(prod_str.strip('[]'), sep=' ')

    with out1:
        fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Import, Export {year}", f"Monthly Production {year}"))

        fig.add_trace(
            go.Bar(x=["Import", "Export"], y=[total_import, total_export], name="Trade"),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],
                y=total_production,
                mode="markers+text",
                name="Production",
                fill='tozeroy',
                fillcolor="rgba(100, 200, 100, 0.3)"
            ),
            row=1, col=2
        )

        fig.update_yaxes(title_text="Quantity", row=1, col=1)
        fig.update_yaxes(title_text="Quantity", row=1, col=2)
        fig.update_layout(height=450, showlegend=False)

        fig.show()
        display(state_dropdown)
        fig3 = go.Figure()
        state = state_dropdown.value
        monthly_state_production = result[state]
        
        fig3.add_scatter(
            x=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
            y=monthly_state_production,
            mode="markers+text",
            name="State Production",
            fill='tozeroy',
            # fillcolor="rgba(100, 200, 100, 0.3)"
        )
        fig3.update_layout(
            title=f"Monthly Production in {state} for {year}",
            yaxis_title="Quantity",
            height=450,
        )
        fig3.show()


def on_year_change(change):
    update_year_plots(change["new"])

year_dropdown.observe(on_year_change, names="value")
display(widgets.HTML("<h2>Copper Data Visualization Dashboard</h2>"))
display(year_dropdown, out1)
update_year_plots(years[0])


HTML(value='<h2>Copper Data Visualization Dashboard</h2>')

Dropdown(description='Year:', options=(np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int6…

Output()